In [50]:
# 数据读取
import pandas as pd
pd.set_option('display.max_columns', None)
family2014 = pd.read_csv('数据/居民出行数据/2014/family_2014.csv',dtype=str)
travel2014 = pd.read_csv('数据/居民出行数据/2014/midtable_2014.csv',dtype=str)
familymember_2014 = pd.read_csv('数据/居民出行数据/2014/family_member_2014.csv',dtype=str)
family2023 = pd.read_csv('数据/居民出行数据/2023/family_total_33169.csv',dtype=str)
travel2023 = pd.read_csv('数据/居民出行数据/2023/midtable_total_33169.csv',dtype=str)
familymember_2023 = pd.read_csv('数据/居民出行数据/2023/familymember_total_33169.csv',dtype=str)
family_cluster = pd.read_csv('数据/family_cluster_new.csv',dtype=str)
## 家庭变量筛选
valid_member_number = familymember_2023.groupby('家庭编号').size().rename('家庭成员数量_real').reset_index()
family2023 = pd.merge(family2023, valid_member_number, on='家庭编号', how='left')
family2023 = family2023[family2023['家庭成员数量'].astype(int) == family2023['家庭成员数量_real']]
valid_family = family2023[['家庭编号']]
familymember_2023 = pd.merge(familymember_2023, valid_family, on='家庭编号', how='inner')
family2023[['家庭成员数量']].value_counts()
## 家庭连续型变量
family2023[['家庭成员数量','家庭工作人口数','机动车数量','脚踏自行车数量','电动自行车数量','摩托车数量','老年代步车数量']]
have_student_family = familymember_2023[familymember_2023['职业'] == '14'].drop_duplicates(['家庭编号'])[['家庭编号']]
have_student_family['have_student'] = 1
family2023 = pd.merge(family2023, have_student_family, on='家庭编号', how='left').fillna({'have_student':0})
## 家庭离散型变量
family2023[['have_student','家庭年收入']]
family2023['家庭年收入'].isna().sum()
## 个人变量筛选
familymember_2023['age'] = 2023 - familymember_2023['出生年份'].astype(int)
familymember_2023['age_group'] = pd.cut(familymember_2023['age'], bins=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100], right=False, labels=['0-5','6-10','11-15','16-20','21-25','26-30','31-35','36-40','41-45','46-50','51-55','56-60','61-65','66-70','71-75','76-80','81-85','86-90','91-95','96-100'])
familymember_2023['age'].max() , familymember_2023['age'].min()
# familymember_2023['age'] = (familymember_2023['age'] - familymember_2023['age'].min()) / (familymember_2023['age'].max() - familymember_2023['age'].min())
## 连续型变量
familymember_2023[['age']]
(familymember_2023[familymember_2023['关系']=='0']['age']).describe()
familymember_2023.loc[familymember_2023['最高学历'].isna(),'最高学历'] = familymember_2023.loc[familymember_2023['最高学历'].isna(),'教育阶段']
## 离散型变量,这里的关系有点不太对，有的户主很小
familymember_2023[['性别','是否有驾照','关系','最高学历','职业']]
familymember_2023['是否有驾照'] = familymember_2023['是否有驾照'].fillna('0')


income_map = {'A':1, 'B':1, 'C':2, 'D':2, 'E':3, 'F':3, 'G':4, 'I':5, 'J':5, 'K':5}
family2023['家庭年收入'] = family2023['家庭年收入'].map(income_map)
familymember_2023['age_group'] = pd.cut(
    familymember_2023['age'],
    bins=range(0, familymember_2023['age'].max() + 6, 5),
    labels=False
)


familymember_2023['age_group'] = familymember_2023['age_group'].fillna(0)
familymember_2023['age'] = familymember_2023['age_group']
# relation_map = {'0':0, '17':1, '1':2, '2':2, '5':2, '6':2, '13':3, '14':3, '15':3, '16':3, '9':3, '10':3, '7':4, '8':4, '11':5, '12':5}
# education_map = {'1':1, '2':1, '3':2, '4':2, '5':3, '6':4, '7':5, '8':6, '9':7}
# occupation_map = {'1':1, '2':1, '3':1, '4':2, '5':2, '6':3, '7':2, '8':3, '9':1, '10':4, '11':4, '12':4, '13':5, '14':6, '15':7, '16':8, '17':8, '18':1, '19':1, '20':8}

# familymember_2023['关系'] = familymember_2023['关系'].map(relation_map)
# familymember_2023['最高学历'] = familymember_2023['最高学历'].map(education_map)
# familymember_2023['职业'] = familymember_2023['职业'].map(occupation_map)
familymember_2023['关系'].value_counts().shape, familymember_2023['最高学历'].value_counts().shape, familymember_2023['职业'].value_counts().shape
from population_data_process_nonclip_reclass import *
## 家庭的变量编码
test = PopulationDataEncoder()
family2023 = pd.merge(family2023,family_cluster[['家庭编号','cluster']], on='家庭编号', how='left')
# 2. 拟合数据 (需要你的实际数据)
test.fit_family_data(family2023)
test.fit_person_data(familymember_2023)

# WGAN测试

In [51]:
from WGAN_baseline.WGAN import WGAN

In [52]:
import torch
import numpy as np
import os
import json
from datetime import datetime
import pandas as pd

n_samples = 33166

checkpoint_path = 'WGAN_baseline/wgan_checkpoints/final_model.pt'  # 模型检查点路径
# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 首先加载配置文件以获取模型参数
config_path = checkpoint_path.replace('.pt', '_config.json')
if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    print("从配置文件加载模型参数:")
    print(json.dumps(config, indent=2))
else:
    # 如果没有配置文件，使用默认参数
    print("未找到配置文件，使用默认参数")
    config = {
        'z_dim': 512,
        'hidden_dim': 512,
        'data_dim': 10 + 51*8,  # 根据你的代码中的设置
        'depth': 3
    }

# 创建WGAN实例
wgan = WGAN(
    z_dim=config.get('z_dim', 512),
    hidden_dim=config.get('hidden_dim', 512),
    data_dim=config.get('data_dim', 418),
    depth=3,  # 注意：原代码中depth参数可能没有保存在config中
    device=device
)

# 加载模型检查点
print(f"\n正在加载模型: {checkpoint_path}")
epoch, iteration = wgan.load_checkpoint(checkpoint_path)

# 打印训练历史的最后几个值（如果存在）
if wgan.train_history and wgan.train_history['epoch']:
    print("\n训练历史摘要:")
    print(f"最后的epoch: {wgan.train_history['epoch'][-1]}")
    print(f"最终Critic Loss: {wgan.train_history['critic_loss'][-1]:.4f}")
    print(f"最终Generator Loss: {wgan.train_history['generator_loss'][-1]:.4f}")
    print(f"最终Wasserstein Distance: {wgan.train_history['wasserstein_distance'][-1]:.4f}")

# 生成数据
print(f"\n正在生成 {n_samples} 个样本...")

# 分批生成以避免内存问题
batch_size = 33169
all_samples = []

with torch.no_grad():
    for i in range(0, n_samples, batch_size):
        current_batch_size = min(batch_size, n_samples - i)
        
        # 生成潜在向量
        z = torch.randn(current_batch_size, wgan.z_dim).to(device)
        
        # 生成样本
        generated_batch = wgan.generate(n_samples=current_batch_size, z=z)
        
        # 转换为CPU并添加到列表
        all_samples.append(generated_batch.cpu().numpy())
        
        print(f"已生成 {i + current_batch_size}/{n_samples} 个样本")

# 合并所有批次
generated_data = np.concatenate(all_samples, axis=0)
print(f"\n生成完成! 数据形状: {generated_data.shape}")


使用设备: cuda
从配置文件加载模型参数:
{
  "z_dim": 512,
  "hidden_dim": 512,
  "data_dim": 418,
  "device": "cuda",
  "n_epochs": 10000,
  "n_critic": 5,
  "clip_value": 0.01,
  "lr": 5e-05,
  "train_start": "2025-11-14T21:06:50.719060"
}

正在加载模型: WGAN_baseline/wgan_checkpoints/final_model.pt
Model loaded from WGAN_baseline/wgan_checkpoints/final_model.pt
Checkpoint from epoch 9999, iteration 320000

训练历史摘要:
最后的epoch: 9999
最终Critic Loss: -0.0371
最终Generator Loss: -0.1497
最终Wasserstein Distance: 0.0371

正在生成 33166 个样本...
已生成 33166/33166 个样本

生成完成! 数据形状: (33166, 418)


In [53]:
samples = generated_data[:, :10]
samples_person = generated_data[:, 10:].reshape(-1, 51)

In [54]:
results_family_df = pd.DataFrame(samples)
results_family_df['have_student'] = results_family_df[[8,9]].values.argmax(axis=1)

results_family_df = results_family_df[[0,1,2,3,4,5,6,7,'have_student']]
results_family_df.columns = ['family_家庭成员数量','family_家庭工作人口数','family_机动车数量','family_脚踏自行车数量','family_电动自行车数量','family_摩托车数量','family_老年代步车数量','income','have_student']

results_family_df.rename(columns = {'income' : 'family_家庭年收入'}, inplace=True)
decode_results_family = test.decode_family_continuous(results_family_df)
for col in decode_results_family.keys():
    results_family_df[f'family_{col}'] = decode_results_family[col]


In [55]:
person_mask = torch.zeros((n_samples, 8), device=device)
for i in range(n_samples):
    num_members = int(results_family_df.iloc[i]['family_家庭成员数量'])
    person_mask[i, :num_members] = 1
person_mask = person_mask.to(torch.bool)
results_person_df = pd.DataFrame(samples_person)
real_person_ls = []
for i in range(len(results_family_df['family_家庭成员数量'])):
    member_num = round(results_family_df.loc[i,'family_家庭成员数量'])
    real_person_ls.append(results_person_df.iloc[i*8: i*8 + member_num, :])
results_person_df = pd.concat(real_person_ls, ignore_index=True)
results_person_df['gender'] = results_person_df[[1,2]].values.argmax(axis=1)
results_person_df['license'] = results_person_df[[3,4]].values.argmax(axis=1)
results_person_df['relation'] = results_person_df.iloc[:,5:21].values.argmax(axis=1)
results_person_df['education'] = results_person_df.iloc[:,21:30].values.argmax(axis=1)
results_person_df['occupation'] = results_person_df.iloc[:,30:50].values.argmax(axis=1)


results_person_df = results_person_df[[0,'gender','license','relation','education','occupation',50]]
results_person_df.columns = ['age','gender','license','relation','education','occupation','label']


results_person_df['age'] = test.decode_person_continuous(results_person_df['age'])['age_actual']


In [56]:
raw_person_df = pd.read_csv('数据/raw_person_df.csv')
raw_family_df = pd.read_csv('数据/raw_family_df.csv')

## 指标计算

In [57]:
max_value_list_family = []
for i in range(len(raw_family_df.columns)):
    raw_max = raw_family_df.iloc[:, i].max()
    results_max = results_family_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_family.append(max_value)

In [58]:
max_value_list_person = []
for i in range(len(raw_person_df.columns)):
    raw_max = raw_person_df.iloc[:, i].max()
    results_max = results_person_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_person.append(max_value)

In [59]:
max_value_list_person

[24, 1, 1, 15, 8, 19]

In [60]:
max_value_list_family

[7, 5, 3, 6, 4, 4, 4, 5, 1]

### 家庭和个人独立计算

In [61]:
import itertools
import numpy as np
import pandas as pd
def calc_srmse(raw_df, synth_df, max_value_list):
    srmse_results = {}

    # 遍历所有可能的属性组合（1维、2维、...）
    for r in range(1, len(max_value_list) + 1):
        number_of_combinations = 0
        srmse_sum = 0.0
        for cols in itertools.combinations(raw_df.columns, r):
            # 计算联合分布频率
            real_counts = raw_df.groupby(list(cols)).size().reset_index(name='count')
            synth_counts = synth_df.groupby(list(cols)).size().reset_index(name='count')

            # 合并并补全缺失组合
            merged = pd.merge(real_counts, synth_counts, on=list(cols), how='outer', suffixes=('_real', '_synth')).fillna(0)

            # 转换为频率分布
            merged['π_real'] = merged['count_real'] / merged['count_real'].sum()
            merged['π_synth'] = merged['count_synth'] / merged['count_synth'].sum()

            # 计算 SRMSE
            N_cnt = len(merged)
            numerator = np.sqrt(((merged['π_synth'] - merged['π_real']) ** 2).sum() / N_cnt)
            denominator = (merged['π_real'].sum()) / N_cnt
            srmse = numerator / denominator if denominator != 0 else np.nan

            srmse_results[cols] = srmse
            srmse_sum += srmse
            number_of_combinations += 1
        avg_srmse = srmse_sum / number_of_combinations if number_of_combinations > 0 else np.nan
        print(f'Average SRMSE for {r}-dimensional combinations: {avg_srmse}')
        srmse_results[f'avg_srmse_{r}_dims'] = avg_srmse

    return srmse_results


# 示例调用
srmse_results = calc_srmse(raw_family_df, results_family_df, max_value_list_family)
srmse_results_person = calc_srmse(raw_person_df, results_person_df, max_value_list_person)

Average SRMSE for 1-dimensional combinations: 0.3902052970362462
Average SRMSE for 2-dimensional combinations: 0.977783891557038
Average SRMSE for 3-dimensional combinations: 1.7503701843841157
Average SRMSE for 4-dimensional combinations: 2.6591745180057567
Average SRMSE for 5-dimensional combinations: 3.6354271931396673
Average SRMSE for 6-dimensional combinations: 4.59916563017902
Average SRMSE for 7-dimensional combinations: 5.4818363219321204
Average SRMSE for 8-dimensional combinations: 6.229551844294846
Average SRMSE for 9-dimensional combinations: 6.799803505624727
Average SRMSE for 1-dimensional combinations: 0.6027326636190539
Average SRMSE for 2-dimensional combinations: 1.6149075570362945
Average SRMSE for 3-dimensional combinations: 3.0281905939688323
Average SRMSE for 4-dimensional combinations: 4.662382753355133
Average SRMSE for 5-dimensional combinations: 6.2600434147134685
Average SRMSE for 6-dimensional combinations: 7.571129609972403


In [62]:
srmse_results

{('family_家庭成员数量',): 0.5989913020709344,
 ('family_家庭工作人口数',): 0.4772369462997983,
 ('family_机动车数量',): 0.07947838970835512,
 ('family_脚踏自行车数量',): 0.3920393896706356,
 ('family_电动自行车数量',): 0.4154671942502576,
 ('family_摩托车数量',): 0.07105409604064791,
 ('family_老年代步车数量',): 0.3296251268630056,
 ('family_家庭年收入',): 1.0719134989405812,
 ('have_student',): 0.07604172948199969,
 'avg_srmse_1_dims': 0.3902052970362462,
 ('family_家庭成员数量', 'family_家庭工作人口数'): 1.4128645387316545,
 ('family_家庭成员数量', 'family_机动车数量'): 0.860313589155126,
 ('family_家庭成员数量', 'family_脚踏自行车数量'): 1.2087574093925557,
 ('family_家庭成员数量', 'family_电动自行车数量'): 1.1957238663065155,
 ('family_家庭成员数量', 'family_摩托车数量'): 1.0392964586133813,
 ('family_家庭成员数量', 'family_老年代步车数量'): 1.274080965478557,
 ('family_家庭成员数量', 'family_家庭年收入'): 2.0158050563441927,
 ('family_家庭成员数量', 'have_student'): 0.7409321054857186,
 ('family_家庭工作人口数', 'family_机动车数量'): 0.8321486247270378,
 ('family_家庭工作人口数', 'family_脚踏自行车数量'): 0.9502656705132825,
 ('family_家庭工作人口数

In [63]:
srmse_results_person

{('age',): 2.4682312245761575,
 ('gender',): 0.004947510611734263,
 ('license',): 0.05911307128954263,
 ('relation',): 0.21487473813797608,
 ('education',): 0.32028873157668875,
 ('occupation',): 0.5489407055222236,
 'avg_srmse_1_dims': 0.6027326636190539,
 ('age', 'gender'): 2.3621761909200263,
 ('age', 'license'): 2.5214287235559913,
 ('age', 'relation'): 4.77645967874745,
 ('age', 'education'): 3.3915292928794387,
 ('age', 'occupation'): 5.276184919275209,
 ('gender', 'license'): 0.06067375151593503,
 ('gender', 'relation'): 0.31821344686988534,
 ('gender', 'education'): 0.32134270495495676,
 ('gender', 'occupation'): 0.5576007619611587,
 ('license', 'relation'): 0.23347354704420747,
 ('license', 'education'): 0.35947627408464183,
 ('license', 'occupation'): 0.676471826978873,
 ('relation', 'education'): 0.7531894446166058,
 ('relation', 'occupation'): 1.411602115218652,
 ('education', 'occupation'): 1.2037906769213862,
 'avg_srmse_2_dims': 1.6149075570362945,
 ('age', 'gender', 'li

### 将个人集成到家庭中

In [64]:
## 有老人、有学生、有驾照的比例、性别比例、本科以上比例、雇佣比例, 集成到家庭中
total_ls_raw = []
total_ls_generate = []
person_number_raw = raw_family_df['family_家庭成员数量'].values.cumsum()
person_number_generate = results_family_df['family_家庭成员数量'].values.cumsum()
for i in range(len(person_number_raw)):
    if i == 0:
        start_idx = 0
    else:
        start_idx = person_number_raw[i-1]
    end_idx = person_number_raw[i]
    person_in_family = raw_person_df.iloc[start_idx:end_idx,:]
    
    ls = []
    ls.append(int((person_in_family['age'] > 11).sum() >= 1))
    ls.append((person_in_family['license']).mean())
    ls.append((person_in_family['gender']).mean())
    ls.append((person_in_family['education'].isin([6, 7])).mean())
    ls.append((-person_in_family['occupation'].isin([14, 13, 16])).mean())
    total_ls_raw.append(ls)

for i in range(len(person_number_generate)):
    if i == 0:
        start_idx = 0
    else:
        start_idx = person_number_generate[i-1]
    end_idx = person_number_generate[i]
    person_in_family_generate = results_person_df.iloc[start_idx:end_idx,:]

    ls_generate = []
    ls_generate.append(int((person_in_family_generate['age'] > 11).sum() >= 1))
    ls_generate.append((person_in_family_generate['license']).mean())
    ls_generate.append((person_in_family_generate['gender']).mean())
    ls_generate.append((person_in_family_generate['education'].isin([6, 7])).mean())
    ls_generate.append((-person_in_family_generate['occupation'].isin([14, 13, 16])).mean())
    total_ls_generate.append(ls_generate)




In [65]:
extended_raw_family_df = pd.concat([raw_family_df.reset_index(drop=True), pd.DataFrame(total_ls_raw, columns=['have_elder_ext', 'license_ext', 'gender_ext', 'education_ext', 'employed_ext'])], axis=1)
extended_results_family_df = pd.concat([results_family_df.reset_index(drop=True), pd.DataFrame(total_ls_generate, columns=['have_elder_ext', 'license_ext', 'gender_ext', 'education_ext', 'employed_ext'])], axis=1)

In [66]:
extended_raw_family_df['license_ext'] = pd.cut(
    extended_raw_family_df['license_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['license_ext'] = pd.cut(
    extended_results_family_df['license_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['gender_ext'] = pd.cut(
    extended_raw_family_df['gender_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['gender_ext'] = pd.cut(
    extended_results_family_df['gender_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['education_ext'] = pd.cut(
    extended_raw_family_df['education_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['education_ext'] = pd.cut(
    extended_results_family_df['education_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['employed_ext'] = pd.cut(
    extended_raw_family_df['employed_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['employed_ext'] = pd.cut(
    extended_results_family_df['employed_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

In [67]:
max_value_list_extendfamily = []
for i in range(len(extended_raw_family_df.columns)):
    raw_max = extended_raw_family_df.iloc[:, i].max()
    results_max = extended_results_family_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_extendfamily.append(max_value)

In [68]:
srmse_results = calc_srmse(extended_raw_family_df, extended_results_family_df, max_value_list_extendfamily)

Average SRMSE for 1-dimensional combinations: 0.3931093538582839
Average SRMSE for 2-dimensional combinations: 0.8855992326667942
Average SRMSE for 3-dimensional combinations: 1.510122144023872
Average SRMSE for 4-dimensional combinations: 2.2597482465664123
Average SRMSE for 5-dimensional combinations: 3.0856975248666663
Average SRMSE for 6-dimensional combinations: 3.9122106287568177
Average SRMSE for 7-dimensional combinations: 4.664743435142189
Average SRMSE for 8-dimensional combinations: 5.285610789408407
Average SRMSE for 9-dimensional combinations: 5.741851030336494
Average SRMSE for 10-dimensional combinations: 6.0274402190864205
Average SRMSE for 11-dimensional combinations: 6.158412522896381
Average SRMSE for 12-dimensional combinations: 6.163050910137712
Average SRMSE for 13-dimensional combinations: 6.07067667165449
Average SRMSE for 14-dimensional combinations: 5.903376361723702


# VAE测试

In [69]:
from VAE_baseline.VAEModel import *

In [70]:
model_path = 'VAE_baseline/vae_best_model.pth'
model = VAE(input_dim=10+51*8, hidden_dim=1024, latent_dim=512)
    
# 加载模型权重
model.load_state_dict(torch.load(model_path, map_location=device))

# 将模型移到指定设备
model = model.to(device)

# 设置为评估模式
model.eval()

VAE(
  (fc1): Linear(in_features=418, out_features=1024, bias=True)
  (fc21): Linear(in_features=1024, out_features=512, bias=True)
  (fc22): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=418, bias=True)
)

In [71]:
z = torch.randn(33166, model.fc3.in_features).to(device)
generated_data_vae = model.decode(z).cpu()

In [72]:
generated_data = generated_data_vae.cpu().detach().numpy()

In [73]:
samples = generated_data[:, :10]
samples_person = generated_data[:, 10:].reshape(-1, 51)

In [74]:
results_family_df = pd.DataFrame(samples)
results_family_df['have_student'] = results_family_df[[8,9]].values.argmax(axis=1)

results_family_df = results_family_df[[0,1,2,3,4,5,6,7,'have_student']]
results_family_df.columns = ['family_家庭成员数量','family_家庭工作人口数','family_机动车数量','family_脚踏自行车数量','family_电动自行车数量','family_摩托车数量','family_老年代步车数量','income','have_student']

results_family_df.rename(columns = {'income' : 'family_家庭年收入'}, inplace=True)
decode_results_family = test.decode_family_continuous(results_family_df)
for col in decode_results_family.keys():
    results_family_df[f'family_{col}'] = decode_results_family[col]


In [75]:
person_mask = torch.zeros((n_samples, 8), device=device)
for i in range(n_samples):
    num_members = int(results_family_df.iloc[i]['family_家庭成员数量'])
    person_mask[i, :num_members] = 1
person_mask = person_mask.to(torch.bool)
results_person_df = pd.DataFrame(samples_person)
real_person_ls = []
for i in range(len(results_family_df['family_家庭成员数量'])):
    member_num = round(results_family_df.loc[i,'family_家庭成员数量'])
    real_person_ls.append(results_person_df.iloc[i*8: i*8 + member_num, :])
results_person_df = pd.concat(real_person_ls, ignore_index=True)
results_person_df['gender'] = results_person_df[[1,2]].values.argmax(axis=1)
results_person_df['license'] = results_person_df[[3,4]].values.argmax(axis=1)
results_person_df['relation'] = results_person_df.iloc[:,5:21].values.argmax(axis=1)
results_person_df['education'] = results_person_df.iloc[:,21:30].values.argmax(axis=1)
results_person_df['occupation'] = results_person_df.iloc[:,30:50].values.argmax(axis=1)


results_person_df = results_person_df[[0,'gender','license','relation','education','occupation',50]]
results_person_df.columns = ['age','gender','license','relation','education','occupation','label']


results_person_df['age'] = test.decode_person_continuous(results_person_df['age'])['age_actual']


## 指标计算

In [76]:
max_value_list_family = []
for i in range(len(raw_family_df.columns)):
    raw_max = raw_family_df.iloc[:, i].max()
    results_max = results_family_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_family.append(max_value)

In [77]:
max_value_list_person = []
for i in range(len(raw_person_df.columns)):
    raw_max = raw_person_df.iloc[:, i].max()
    results_max = results_person_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_person.append(max_value)

In [78]:
max_value_list_person

[24, 1, 1, 15, 8, 19]

In [79]:
max_value_list_family

[7, 5, 3, 6, 4, 4, 4, 5, 1]

### 家庭和个人独立计算

In [80]:
import itertools
import numpy as np
import pandas as pd
def calc_srmse(raw_df, synth_df, max_value_list):
    srmse_results = {}

    # 遍历所有可能的属性组合（1维、2维、...）
    for r in range(1, len(max_value_list) + 1):
        number_of_combinations = 0
        srmse_sum = 0.0
        for cols in itertools.combinations(raw_df.columns, r):
            # 计算联合分布频率
            real_counts = raw_df.groupby(list(cols)).size().reset_index(name='count')
            synth_counts = synth_df.groupby(list(cols)).size().reset_index(name='count')

            # 合并并补全缺失组合
            merged = pd.merge(real_counts, synth_counts, on=list(cols), how='outer', suffixes=('_real', '_synth')).fillna(0)

            # 转换为频率分布
            merged['π_real'] = merged['count_real'] / merged['count_real'].sum()
            merged['π_synth'] = merged['count_synth'] / merged['count_synth'].sum()

            # 计算 SRMSE
            N_cnt = len(merged)
            numerator = np.sqrt(((merged['π_synth'] - merged['π_real']) ** 2).sum() / N_cnt)
            denominator = (merged['π_real'].sum()) / N_cnt
            srmse = numerator / denominator if denominator != 0 else np.nan

            srmse_results[cols] = srmse
            srmse_sum += srmse
            number_of_combinations += 1
        avg_srmse = srmse_sum / number_of_combinations if number_of_combinations > 0 else np.nan
        print(f'Average SRMSE for {r}-dimensional combinations: {avg_srmse}')
        srmse_results[f'avg_srmse_{r}_dims'] = avg_srmse

    return srmse_results


# 示例调用
srmse_results = calc_srmse(raw_family_df, results_family_df, max_value_list_family)
srmse_results_person = calc_srmse(raw_person_df, results_person_df, max_value_list_person)

Average SRMSE for 1-dimensional combinations: 0.8513037817418505
Average SRMSE for 2-dimensional combinations: 2.2520629578126194
Average SRMSE for 3-dimensional combinations: 4.207389013448126
Average SRMSE for 4-dimensional combinations: 6.709779490243865
Average SRMSE for 5-dimensional combinations: 9.709074260690379
Average SRMSE for 6-dimensional combinations: 13.069979276309658
Average SRMSE for 7-dimensional combinations: 16.625855472808333
Average SRMSE for 8-dimensional combinations: 20.204232796948492
Average SRMSE for 9-dimensional combinations: 23.621142162578607
Average SRMSE for 1-dimensional combinations: 0.5057536967503312
Average SRMSE for 2-dimensional combinations: 1.2675518335294824
Average SRMSE for 3-dimensional combinations: 2.2754297788707945
Average SRMSE for 4-dimensional combinations: 3.3836327812947564
Average SRMSE for 5-dimensional combinations: 4.4214839684371965
Average SRMSE for 6-dimensional combinations: 5.2613291054756886


In [81]:
srmse_results

{('family_家庭成员数量',): 0.5632402238082752,
 ('family_家庭工作人口数',): 0.5073290024516043,
 ('family_机动车数量',): 1.48043448170669,
 ('family_脚踏自行车数量',): 2.2681057729326874,
 ('family_电动自行车数量',): 1.2096121054539504,
 ('family_摩托车数量',): 0.07105409604064791,
 ('family_老年代步车数量',): 0.3296251268630056,
 ('family_家庭年收入',): 1.2038703427437398,
 ('have_student',): 0.028462883676053846,
 'avg_srmse_1_dims': 0.8513037817418505,
 ('family_家庭成员数量', 'family_家庭工作人口数'): 1.4944292996131605,
 ('family_家庭成员数量', 'family_机动车数量'): 2.7392865718008887,
 ('family_家庭成员数量', 'family_脚踏自行车数量'): 3.700696097059794,
 ('family_家庭成员数量', 'family_电动自行车数量'): 2.3218528881472924,
 ('family_家庭成员数量', 'family_摩托车数量'): 0.9861504917752751,
 ('family_家庭成员数量', 'family_老年代步车数量'): 1.2309101063688466,
 ('family_家庭成员数量', 'family_家庭年收入'): 2.527107988465466,
 ('family_家庭成员数量', 'have_student'): 0.7217245013624669,
 ('family_家庭工作人口数', 'family_机动车数量'): 2.308155841355375,
 ('family_家庭工作人口数', 'family_脚踏自行车数量'): 3.2611298557140835,
 ('family_家庭工作人口数', 

In [82]:
srmse_results_person

{('age',): 2.4875545200543674,
 ('gender',): 0.03326460906263162,
 ('license',): 0.050061968016946234,
 ('relation',): 0.16028086376287323,
 ('education',): 0.1266749368651544,
 ('occupation',): 0.17668528274001447,
 'avg_srmse_1_dims': 0.5057536967503312,
 ('age', 'gender'): 2.386322581111879,
 ('age', 'license'): 2.6594042524250345,
 ('age', 'relation'): 4.245050909385812,
 ('age', 'education'): 3.2254331780386813,
 ('age', 'occupation'): 4.16275015599886,
 ('gender', 'license'): 0.0642923060520375,
 ('gender', 'relation'): 0.20140004465786496,
 ('gender', 'education'): 0.13625317123112385,
 ('gender', 'occupation'): 0.1921115996311895,
 ('license', 'relation'): 0.1845126357101064,
 ('license', 'education'): 0.197446992906809,
 ('license', 'occupation'): 0.25674156997715286,
 ('relation', 'education'): 0.3122412863594432,
 ('relation', 'occupation'): 0.39223090288783297,
 ('education', 'occupation'): 0.39708591656840575,
 'avg_srmse_2_dims': 1.2675518335294824,
 ('age', 'gender', 'li

### 将个人集成到家庭中

In [83]:
## 有老人、有学生、有驾照的比例、性别比例、本科以上比例、雇佣比例, 集成到家庭中
total_ls_raw = []
total_ls_generate = []
person_number_raw = raw_family_df['family_家庭成员数量'].values.cumsum()
person_number_generate = results_family_df['family_家庭成员数量'].values.cumsum()
for i in range(len(person_number_raw)):
    if i == 0:
        start_idx = 0
    else:
        start_idx = person_number_raw[i-1]
    end_idx = person_number_raw[i]
    person_in_family = raw_person_df.iloc[start_idx:end_idx,:]
    
    ls = []
    ls.append(int((person_in_family['age'] > 11).sum() >= 1))
    ls.append((person_in_family['license']).mean())
    ls.append((person_in_family['gender']).mean())
    ls.append((person_in_family['education'].isin([6, 7])).mean())
    ls.append((-person_in_family['occupation'].isin([14, 13, 16])).mean())
    total_ls_raw.append(ls)

for i in range(len(person_number_generate)):
    if i == 0:
        start_idx = 0
    else:
        start_idx = person_number_generate[i-1]
    end_idx = person_number_generate[i]
    person_in_family_generate = results_person_df.iloc[start_idx:end_idx,:]

    ls_generate = []
    ls_generate.append(int((person_in_family_generate['age'] > 11).sum() >= 1))
    ls_generate.append((person_in_family_generate['license']).mean())
    ls_generate.append((person_in_family_generate['gender']).mean())
    ls_generate.append((person_in_family_generate['education'].isin([6, 7])).mean())
    ls_generate.append((-person_in_family_generate['occupation'].isin([14, 13, 16])).mean())
    total_ls_generate.append(ls_generate)




In [84]:
extended_raw_family_df = pd.concat([raw_family_df.reset_index(drop=True), pd.DataFrame(total_ls_raw, columns=['have_elder_ext', 'license_ext', 'gender_ext', 'education_ext', 'employed_ext'])], axis=1)
extended_results_family_df = pd.concat([results_family_df.reset_index(drop=True), pd.DataFrame(total_ls_generate, columns=['have_elder_ext', 'license_ext', 'gender_ext', 'education_ext', 'employed_ext'])], axis=1)

In [85]:
extended_raw_family_df['license_ext'] = pd.cut(
    extended_raw_family_df['license_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['license_ext'] = pd.cut(
    extended_results_family_df['license_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['gender_ext'] = pd.cut(
    extended_raw_family_df['gender_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['gender_ext'] = pd.cut(
    extended_results_family_df['gender_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['education_ext'] = pd.cut(
    extended_raw_family_df['education_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['education_ext'] = pd.cut(
    extended_results_family_df['education_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['employed_ext'] = pd.cut(
    extended_raw_family_df['employed_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['employed_ext'] = pd.cut(
    extended_results_family_df['employed_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

In [86]:
max_value_list_extendfamily = []
for i in range(len(extended_raw_family_df.columns)):
    raw_max = extended_raw_family_df.iloc[:, i].max()
    results_max = extended_results_family_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_extendfamily.append(max_value)

In [87]:
srmse_results = calc_srmse(extended_raw_family_df, extended_results_family_df, max_value_list_extendfamily)

Average SRMSE for 1-dimensional combinations: 0.6220259272072841
Average SRMSE for 2-dimensional combinations: 1.5040311062071876
Average SRMSE for 3-dimensional combinations: 2.687868012185195
Average SRMSE for 4-dimensional combinations: 4.195208370127212
Average SRMSE for 5-dimensional combinations: 5.982603979683436
Average SRMSE for 6-dimensional combinations: 7.9253941345504675
Average SRMSE for 7-dimensional combinations: 9.865091516681098
Average SRMSE for 8-dimensional combinations: 11.644903833743015
Average SRMSE for 9-dimensional combinations: 13.13615067151749
Average SRMSE for 10-dimensional combinations: 14.258748806678756
Average SRMSE for 11-dimensional combinations: 14.987338297792164
Average SRMSE for 12-dimensional combinations: 15.342791990654042
Average SRMSE for 13-dimensional combinations: 15.37423160702651
Average SRMSE for 14-dimensional combinations: 15.140873262972464


# CWGAN测试

In [88]:
from WCGAN_baseline.WCGAN import CWGAN

In [89]:
import torch
import numpy as np
import os
import json
from datetime import datetime
import pandas as pd

cluster_profile = pd.read_csv('数据/cluster_profile_improved.csv')
cluster_info = torch.from_numpy(family2023['cluster'].values.astype(int))
cluster_profile_generate = []
for i in cluster_info.numpy():
    tmp = cluster_profile[cluster_profile['improved_cluster']==i]
    profile = tmp.values[0][1:]
    cluster_profile_generate.append(profile)
cluster_profile_generate = torch.tensor(cluster_profile_generate).to(torch.float32)
n_samples = 33166

checkpoint_path = 'WCGAN_baseline/cwgan_checkpoints/final_model.pt'  # 模型检查点路径
# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 首先加载配置文件以获取模型参数
config_path = checkpoint_path.replace('.pt', '_config.json')
if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    print("从配置文件加载模型参数:")
    print(json.dumps(config, indent=2))
else:
    # 如果没有配置文件，使用默认参数
    print("未找到配置文件，使用默认参数")
    config = {
        'z_dim': 512,
        'hidden_dim': 512,
        'data_dim': 10 + 51*8,  # 根据你的代码中的设置
        'depth': 3
    }

# 创建WGAN实例
cwgan = CWGAN(
    z_dim=config.get('z_dim', 512),
    hidden_dim=config.get('hidden_dim', 512),
    data_dim=config.get('data_dim', 418),
    depth=3,  # 注意：原代码中depth参数可能没有保存在config中
    device=device
)

# 加载模型检查点
print(f"\n正在加载模型: {checkpoint_path}")
epoch, iteration = cwgan.load_checkpoint(checkpoint_path)

# 打印训练历史的最后几个值（如果存在）
if cwgan.train_history and cwgan.train_history['epoch']:
    print("\n训练历史摘要:")
    print(f"最后的epoch: {cwgan.train_history['epoch'][-1]}")
    print(f"最终Critic Loss: {cwgan.train_history['critic_loss'][-1]:.4f}")
    print(f"最终Generator Loss: {cwgan.train_history['generator_loss'][-1]:.4f}")
    print(f"最终Wasserstein Distance: {cwgan.train_history['wasserstein_distance'][-1]:.4f}")

# 生成数据
print(f"\n正在生成 {n_samples} 个样本...")

# 分批生成以避免内存问题
batch_size = 33169
all_samples = []

with torch.no_grad():
    for i in range(0, n_samples, batch_size):
        current_batch_size = min(batch_size, n_samples - i)
        
        # 生成潜在向量
        z = torch.randn(current_batch_size, cwgan.z_dim).to(device)
        
        # 生成样本
        generated_batch = cwgan.generate(cluster_info, cluster_profile_generate, n_samples=current_batch_size, z=z)
        
        # 转换为CPU并添加到列表
        all_samples.append(generated_batch.cpu().numpy())
        
        print(f"已生成 {i + current_batch_size}/{n_samples} 个样本")

# 合并所有批次
generated_data = np.concatenate(all_samples, axis=0)
print(f"\n生成完成! 数据形状: {generated_data.shape}")


使用设备: cuda
从配置文件加载模型参数:
{
  "z_dim": 512,
  "hidden_dim": 512,
  "data_dim": 418,
  "num_classes": 49,
  "class_dropout_prob": 0.1,
  "device": "cuda",
  "n_epochs": 10000,
  "n_critic": 5,
  "clip_value": 0.01,
  "lr": 5e-05,
  "train_start": "2025-11-16T14:41:52.260247"
}

正在加载模型: WCGAN_baseline/cwgan_checkpoints/final_model.pt
Model loaded from WCGAN_baseline/cwgan_checkpoints/final_model.pt
Checkpoint from epoch 9999, iteration 320000

训练历史摘要:
最后的epoch: 10000
最终Critic Loss: -0.0465
最终Generator Loss: -0.2068
最终Wasserstein Distance: 0.0465

正在生成 33166 个样本...
已生成 33166/33166 个样本

生成完成! 数据形状: (33166, 418)


In [90]:
samples = generated_data[:, :10]
samples_person = generated_data[:, 10:].reshape(-1, 51)

In [91]:
results_family_df = pd.DataFrame(samples)
results_family_df['have_student'] = results_family_df[[8,9]].values.argmax(axis=1)

results_family_df = results_family_df[[0,1,2,3,4,5,6,7,'have_student']]
results_family_df.columns = ['family_家庭成员数量','family_家庭工作人口数','family_机动车数量','family_脚踏自行车数量','family_电动自行车数量','family_摩托车数量','family_老年代步车数量','income','have_student']

results_family_df.rename(columns = {'income' : 'family_家庭年收入'}, inplace=True)
decode_results_family = test.decode_family_continuous(results_family_df)
for col in decode_results_family.keys():
    results_family_df[f'family_{col}'] = decode_results_family[col]


In [92]:
person_mask = torch.zeros((n_samples, 8), device=device)
for i in range(n_samples):
    num_members = int(results_family_df.iloc[i]['family_家庭成员数量'])
    person_mask[i, :num_members] = 1
person_mask = person_mask.to(torch.bool)
results_person_df = pd.DataFrame(samples_person)
real_person_ls = []
for i in range(len(results_family_df['family_家庭成员数量'])):
    member_num = round(results_family_df.loc[i,'family_家庭成员数量'])
    real_person_ls.append(results_person_df.iloc[i*8: i*8 + member_num, :])
results_person_df = pd.concat(real_person_ls, ignore_index=True)
results_person_df['gender'] = results_person_df[[1,2]].values.argmax(axis=1)
results_person_df['license'] = results_person_df[[3,4]].values.argmax(axis=1)
results_person_df['relation'] = results_person_df.iloc[:,5:21].values.argmax(axis=1)
results_person_df['education'] = results_person_df.iloc[:,21:30].values.argmax(axis=1)
results_person_df['occupation'] = results_person_df.iloc[:,30:50].values.argmax(axis=1)


results_person_df = results_person_df[[0,'gender','license','relation','education','occupation',50]]
results_person_df.columns = ['age','gender','license','relation','education','occupation','label']


results_person_df['age'] = test.decode_person_continuous(results_person_df['age'])['age_actual']


In [93]:
raw_person_df = pd.read_csv('数据/raw_person_df.csv')
raw_family_df = pd.read_csv('数据/raw_family_df.csv')

## 指标计算

In [94]:
max_value_list_family = []
for i in range(len(raw_family_df.columns)):
    raw_max = raw_family_df.iloc[:, i].max()
    results_max = results_family_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_family.append(max_value)

In [95]:
max_value_list_person = []
for i in range(len(raw_person_df.columns)):
    raw_max = raw_person_df.iloc[:, i].max()
    results_max = results_person_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_person.append(max_value)

In [96]:
max_value_list_person

[24, 1, 1, 15, 8, 19]

In [97]:
max_value_list_family

[7, 5, 3, 6, 4, 4, 4, 5, 1]

### 家庭和个人独立计算

In [98]:
import itertools
import numpy as np
import pandas as pd
def calc_srmse(raw_df, synth_df, max_value_list):
    srmse_results = {}

    # 遍历所有可能的属性组合（1维、2维、...）
    for r in range(1, len(max_value_list) + 1):
        number_of_combinations = 0
        srmse_sum = 0.0
        for cols in itertools.combinations(raw_df.columns, r):
            # 计算联合分布频率
            real_counts = raw_df.groupby(list(cols)).size().reset_index(name='count')
            synth_counts = synth_df.groupby(list(cols)).size().reset_index(name='count')

            # 合并并补全缺失组合
            merged = pd.merge(real_counts, synth_counts, on=list(cols), how='outer', suffixes=('_real', '_synth')).fillna(0)

            # 转换为频率分布
            merged['π_real'] = merged['count_real'] / merged['count_real'].sum()
            merged['π_synth'] = merged['count_synth'] / merged['count_synth'].sum()

            # 计算 SRMSE
            N_cnt = len(merged)
            numerator = np.sqrt(((merged['π_synth'] - merged['π_real']) ** 2).sum() / N_cnt)
            denominator = (merged['π_real'].sum()) / N_cnt
            srmse = numerator / denominator if denominator != 0 else np.nan

            srmse_results[cols] = srmse
            srmse_sum += srmse
            number_of_combinations += 1
        avg_srmse = srmse_sum / number_of_combinations if number_of_combinations > 0 else np.nan
        print(f'Average SRMSE for {r}-dimensional combinations: {avg_srmse}')
        srmse_results[f'avg_srmse_{r}_dims'] = avg_srmse

    return srmse_results


# 示例调用
srmse_results = calc_srmse(raw_family_df, results_family_df, max_value_list_family)
srmse_results_person = calc_srmse(raw_person_df, results_person_df, max_value_list_person)

Average SRMSE for 1-dimensional combinations: 0.43860598224555286
Average SRMSE for 2-dimensional combinations: 1.1214056091679983
Average SRMSE for 3-dimensional combinations: 2.0596568898163774
Average SRMSE for 4-dimensional combinations: 3.220779531495104
Average SRMSE for 5-dimensional combinations: 4.54710598010092
Average SRMSE for 6-dimensional combinations: 5.950196336558498
Average SRMSE for 7-dimensional combinations: 7.336604150391498
Average SRMSE for 8-dimensional combinations: 8.616107109283435
Average SRMSE for 9-dimensional combinations: 9.70164731473805
Average SRMSE for 1-dimensional combinations: 0.5219402369568742
Average SRMSE for 2-dimensional combinations: 1.3719588690869662
Average SRMSE for 3-dimensional combinations: 2.5723355437186006
Average SRMSE for 4-dimensional combinations: 4.047585752358588
Average SRMSE for 5-dimensional combinations: 5.612011571289366
Average SRMSE for 6-dimensional combinations: 7.0611852560202255


In [99]:
srmse_results

{('family_家庭成员数量',): 0.6607056147722385,
 ('family_家庭工作人口数',): 0.5879099433764119,
 ('family_机动车数量',): 0.2911624258495894,
 ('family_脚踏自行车数量',): 0.224647396177668,
 ('family_电动自行车数量',): 0.691871192365918,
 ('family_摩托车数量',): 0.07105409604064791,
 ('family_老年代步车数量',): 0.3296251268630056,
 ('family_家庭年收入',): 1.0533315694584606,
 ('have_student',): 0.037146475306036236,
 'avg_srmse_1_dims': 0.43860598224555286,
 ('family_家庭成员数量', 'family_家庭工作人口数'): 1.6498957441257567,
 ('family_家庭成员数量', 'family_机动车数量'): 1.132644894905896,
 ('family_家庭成员数量', 'family_脚踏自行车数量'): 1.2494295710461474,
 ('family_家庭成员数量', 'family_电动自行车数量'): 1.5855630959567324,
 ('family_家庭成员数量', 'family_摩托车数量'): 1.1520818886618023,
 ('family_家庭成员数量', 'family_老年代步车数量'): 1.3850511607296179,
 ('family_家庭成员数量', 'family_家庭年收入'): 2.15035331502326,
 ('family_家庭成员数量', 'have_student'): 0.9297355353823025,
 ('family_家庭工作人口数', 'family_机动车数量'): 1.1557423764977013,
 ('family_家庭工作人口数', 'family_脚踏自行车数量'): 1.071339756783479,
 ('family_家庭工作人口数', 

In [100]:
srmse_results_person

{('age',): 2.0829057316331903,
 ('gender',): 0.0030810291551370828,
 ('license',): 0.12958905704250318,
 ('relation',): 0.36348058679336087,
 ('education',): 0.2711461232671392,
 ('occupation',): 0.28143889384991416,
 'avg_srmse_1_dims': 0.5219402369568742,
 ('age', 'gender'): 1.9918300862276974,
 ('age', 'license'): 2.1273122589068083,
 ('age', 'relation'): 4.078411001228732,
 ('age', 'education'): 2.8865165847496477,
 ('age', 'occupation'): 3.805585394852696,
 ('gender', 'license'): 0.1545557331138189,
 ('gender', 'relation'): 0.5492494965325034,
 ('gender', 'education'): 0.28902690127441605,
 ('gender', 'occupation'): 0.3243077977730731,
 ('license', 'relation'): 0.5245396672107938,
 ('license', 'education'): 0.38493497940202237,
 ('license', 'occupation'): 0.4805821889842793,
 ('relation', 'education'): 1.0142135766958684,
 ('relation', 'occupation'): 1.0221503873497273,
 ('education', 'occupation'): 0.9461669820024114,
 'avg_srmse_2_dims': 1.3719588690869662,
 ('age', 'gender', 'l

### 将个人集成到家庭中

In [101]:
## 有老人、有学生、有驾照的比例、性别比例、本科以上比例、雇佣比例, 集成到家庭中
total_ls_raw = []
total_ls_generate = []
person_number_raw = raw_family_df['family_家庭成员数量'].values.cumsum()
person_number_generate = results_family_df['family_家庭成员数量'].values.cumsum()
for i in range(len(person_number_raw)):
    if i == 0:
        start_idx = 0
    else:
        start_idx = person_number_raw[i-1]
    end_idx = person_number_raw[i]
    person_in_family = raw_person_df.iloc[start_idx:end_idx,:]
    
    ls = []
    ls.append(int((person_in_family['age'] > 11).sum() >= 1))
    ls.append((person_in_family['license']).mean())
    ls.append((person_in_family['gender']).mean())
    ls.append((person_in_family['education'].isin([6, 7])).mean())
    ls.append((-person_in_family['occupation'].isin([14, 13, 16])).mean())
    total_ls_raw.append(ls)

for i in range(len(person_number_generate)):
    if i == 0:
        start_idx = 0
    else:
        start_idx = person_number_generate[i-1]
    end_idx = person_number_generate[i]
    person_in_family_generate = results_person_df.iloc[start_idx:end_idx,:]

    ls_generate = []
    ls_generate.append(int((person_in_family_generate['age'] > 11).sum() >= 1))
    ls_generate.append((person_in_family_generate['license']).mean())
    ls_generate.append((person_in_family_generate['gender']).mean())
    ls_generate.append((person_in_family_generate['education'].isin([6, 7])).mean())
    ls_generate.append((-person_in_family_generate['occupation'].isin([14, 13, 16])).mean())
    total_ls_generate.append(ls_generate)




In [102]:
extended_raw_family_df = pd.concat([raw_family_df.reset_index(drop=True), pd.DataFrame(total_ls_raw, columns=['have_elder_ext', 'license_ext', 'gender_ext', 'education_ext', 'employed_ext'])], axis=1)
extended_results_family_df = pd.concat([results_family_df.reset_index(drop=True), pd.DataFrame(total_ls_generate, columns=['have_elder_ext', 'license_ext', 'gender_ext', 'education_ext', 'employed_ext'])], axis=1)

In [103]:
extended_raw_family_df['license_ext'] = pd.cut(
    extended_raw_family_df['license_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['license_ext'] = pd.cut(
    extended_results_family_df['license_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['gender_ext'] = pd.cut(
    extended_raw_family_df['gender_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['gender_ext'] = pd.cut(
    extended_results_family_df['gender_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['education_ext'] = pd.cut(
    extended_raw_family_df['education_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['education_ext'] = pd.cut(
    extended_results_family_df['education_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['employed_ext'] = pd.cut(
    extended_raw_family_df['employed_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['employed_ext'] = pd.cut(
    extended_results_family_df['employed_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

In [104]:
max_value_list_extendfamily = []
for i in range(len(extended_raw_family_df.columns)):
    raw_max = extended_raw_family_df.iloc[:, i].max()
    results_max = extended_results_family_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_extendfamily.append(max_value)

In [105]:
srmse_results = calc_srmse(extended_raw_family_df, extended_results_family_df, max_value_list_extendfamily)

Average SRMSE for 1-dimensional combinations: 0.37120540711012817
Average SRMSE for 2-dimensional combinations: 0.8842494081591082
Average SRMSE for 3-dimensional combinations: 1.590336862647833
Average SRMSE for 4-dimensional combinations: 2.4905789528968874
Average SRMSE for 5-dimensional combinations: 3.5420810316942313
Average SRMSE for 6-dimensional combinations: 4.665362123310108
Average SRMSE for 7-dimensional combinations: 5.772639325977482
Average SRMSE for 8-dimensional combinations: 6.7860367773918675
Average SRMSE for 9-dimensional combinations: 7.648609441792396
Average SRMSE for 10-dimensional combinations: 8.330324449196441
Average SRMSE for 11-dimensional combinations: 8.826602166107822
Average SRMSE for 12-dimensional combinations: 9.150338119002592
Average SRMSE for 13-dimensional combinations: 9.320640344958122
Average SRMSE for 14-dimensional combinations: 9.3531994409106


# CVAE测试

In [106]:
from CVAE_baseline.CVAEModel import *

In [107]:
model_path = 'CVAE_baseline/cvae_best_model.pth'
latent_dim = 512  # 潜在空间维度
family_feature = 10
person_feature = 51
max_nodes = 8
person_flat_dim = person_feature * max_nodes
model = CVAE(family_dim=family_feature, 
                 person_flat_dim=person_flat_dim,
                 hidden_dim=1024, 
                 latent_dim=latent_dim)
    
# 加载模型权重
model.load_state_dict(torch.load(model_path, map_location=device))

# 将模型移到指定设备
model = model.to(device)

# 设置为评估模式
model.eval()

CVAE(
  (y_embedder): LabelEmbedder(
    (embedding_table): Embedding(50, 1024)
  )
  (profile_proj_family): Sequential(
    (0): Linear(in_features=69, out_features=1024, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (cluster_norm_family): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=1442, out_features=1024, bias=True)
  (fc21): Linear(in_features=1024, out_features=512, bias=True)
  (fc22): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=1536, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=418, bias=True)
)

In [108]:
cluster_embed = model.y_embedder(cluster_info.to(device), model.training)
profile_embed = model.profile_proj_family(cluster_profile_generate.to(device))
cluster_embed = cluster_embed + profile_embed
cluster_embed = model.cluster_norm_family(cluster_embed)

In [109]:
z = torch.randn(33166, latent_dim).to(device)
generated_data_vae = model.decode(z, cluster_embed).cpu()

In [110]:
generated_data = generated_data_vae.cpu().detach().numpy()

In [111]:
samples = generated_data[:, :10]
samples_person = generated_data[:, 10:].reshape(-1, 51)

In [112]:
results_family_df = pd.DataFrame(samples)
results_family_df['have_student'] = results_family_df[[8,9]].values.argmax(axis=1)

results_family_df = results_family_df[[0,1,2,3,4,5,6,7,'have_student']]
results_family_df.columns = ['family_家庭成员数量','family_家庭工作人口数','family_机动车数量','family_脚踏自行车数量','family_电动自行车数量','family_摩托车数量','family_老年代步车数量','income','have_student']

results_family_df.rename(columns = {'income' : 'family_家庭年收入'}, inplace=True)
decode_results_family = test.decode_family_continuous(results_family_df)
for col in decode_results_family.keys():
    results_family_df[f'family_{col}'] = decode_results_family[col]


In [113]:
person_mask = torch.zeros((n_samples, 8), device=device)
for i in range(n_samples):
    num_members = int(results_family_df.iloc[i]['family_家庭成员数量'])
    person_mask[i, :num_members] = 1
person_mask = person_mask.to(torch.bool)
results_person_df = pd.DataFrame(samples_person)
real_person_ls = []
for i in range(len(results_family_df['family_家庭成员数量'])):
    member_num = round(results_family_df.loc[i,'family_家庭成员数量'])
    real_person_ls.append(results_person_df.iloc[i*8: i*8 + member_num, :])
results_person_df = pd.concat(real_person_ls, ignore_index=True)
results_person_df['gender'] = results_person_df[[1,2]].values.argmax(axis=1)
results_person_df['license'] = results_person_df[[3,4]].values.argmax(axis=1)
results_person_df['relation'] = results_person_df.iloc[:,5:21].values.argmax(axis=1)
results_person_df['education'] = results_person_df.iloc[:,21:30].values.argmax(axis=1)
results_person_df['occupation'] = results_person_df.iloc[:,30:50].values.argmax(axis=1)


results_person_df = results_person_df[[0,'gender','license','relation','education','occupation',50]]
results_person_df.columns = ['age','gender','license','relation','education','occupation','label']


results_person_df['age'] = test.decode_person_continuous(results_person_df['age'])['age_actual']


## 指标计算

In [114]:
max_value_list_family = []
for i in range(len(raw_family_df.columns)):
    raw_max = raw_family_df.iloc[:, i].max()
    results_max = results_family_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_family.append(max_value)

In [115]:
max_value_list_person = []
for i in range(len(raw_person_df.columns)):
    raw_max = raw_person_df.iloc[:, i].max()
    results_max = results_person_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_person.append(max_value)

In [116]:
max_value_list_person

[24, 1, 1, 15, 8, 19]

In [117]:
max_value_list_family

[7, 5, 3, 6, 4, 4, 4, 5, 1]

### 家庭和个人独立计算

In [118]:
import itertools
import numpy as np
import pandas as pd
def calc_srmse(raw_df, synth_df, max_value_list):
    srmse_results = {}

    # 遍历所有可能的属性组合（1维、2维、...）
    for r in range(1, len(max_value_list) + 1):
        number_of_combinations = 0
        srmse_sum = 0.0
        for cols in itertools.combinations(raw_df.columns, r):
            # 计算联合分布频率
            real_counts = raw_df.groupby(list(cols)).size().reset_index(name='count')
            synth_counts = synth_df.groupby(list(cols)).size().reset_index(name='count')

            # 合并并补全缺失组合
            merged = pd.merge(real_counts, synth_counts, on=list(cols), how='outer', suffixes=('_real', '_synth')).fillna(0)

            # 转换为频率分布
            merged['π_real'] = merged['count_real'] / merged['count_real'].sum()
            merged['π_synth'] = merged['count_synth'] / merged['count_synth'].sum()

            # 计算 SRMSE
            N_cnt = len(merged)
            numerator = np.sqrt(((merged['π_synth'] - merged['π_real']) ** 2).sum() / N_cnt)
            denominator = (merged['π_real'].sum()) / N_cnt
            srmse = numerator / denominator if denominator != 0 else np.nan

            srmse_results[cols] = srmse
            srmse_sum += srmse
            number_of_combinations += 1
        avg_srmse = srmse_sum / number_of_combinations if number_of_combinations > 0 else np.nan
        print(f'Average SRMSE for {r}-dimensional combinations: {avg_srmse}')
        srmse_results[f'avg_srmse_{r}_dims'] = avg_srmse

    return srmse_results


# 示例调用
srmse_results = calc_srmse(raw_family_df, results_family_df, max_value_list_family)
srmse_results_person = calc_srmse(raw_person_df, results_person_df, max_value_list_person)

Average SRMSE for 1-dimensional combinations: 0.8963785398248724
Average SRMSE for 2-dimensional combinations: 2.380525034123205
Average SRMSE for 3-dimensional combinations: 4.493113569148923
Average SRMSE for 4-dimensional combinations: 7.256759712788198
Average SRMSE for 5-dimensional combinations: 10.650812108852637
Average SRMSE for 6-dimensional combinations: 14.561796090551525
Average SRMSE for 7-dimensional combinations: 18.835546091834114
Average SRMSE for 8-dimensional combinations: 23.299131560821365
Average SRMSE for 9-dimensional combinations: 27.74754666958433
Average SRMSE for 1-dimensional combinations: 0.6595558379637183
Average SRMSE for 2-dimensional combinations: 1.553329771440457
Average SRMSE for 3-dimensional combinations: 2.650742052743074
Average SRMSE for 4-dimensional combinations: 3.7899997801585816
Average SRMSE for 5-dimensional combinations: 4.801512412959872
Average SRMSE for 6-dimensional combinations: 5.615886916537009


In [119]:
srmse_results

{('family_家庭成员数量',): 0.8462926816837756,
 ('family_家庭工作人口数',): 0.5317381254526754,
 ('family_机动车数量',): 1.48043448170669,
 ('family_脚踏自行车数量',): 2.2681057729326874,
 ('family_电动自行车数量',): 1.2096121054539504,
 ('family_摩托车数量',): 0.07105409604064791,
 ('family_老年代步车数量',): 0.3296251268630056,
 ('family_家庭年收入',): 1.2902622515624453,
 ('have_student',): 0.040282216727974385,
 'avg_srmse_1_dims': 0.8963785398248724,
 ('family_家庭成员数量', 'family_家庭工作人口数'): 1.896806266390539,
 ('family_家庭成员数量', 'family_机动车数量'): 3.1606056626361663,
 ('family_家庭成员数量', 'family_脚踏自行车数量'): 4.157651972418811,
 ('family_家庭成员数量', 'family_电动自行车数量'): 2.7727618953115782,
 ('family_家庭成员数量', 'family_摩托车数量'): 1.4626181188838205,
 ('family_家庭成员数量', 'family_老年代步车数量'): 1.6803150800197573,
 ('family_家庭成员数量', 'family_家庭年收入'): 3.163642205655417,
 ('family_家庭成员数量', 'have_student'): 1.1791559661979278,
 ('family_家庭工作人口数', 'family_机动车数量'): 2.3310458271284618,
 ('family_家庭工作人口数', 'family_脚踏自行车数量'): 3.274267483560018,
 ('family_家庭工作人口数', '

In [120]:
srmse_results_person

{('age',): 2.727633412614639,
 ('gender',): 0.033666407308053425,
 ('license',): 0.050057465897858366,
 ('relation',): 0.4582635732063034,
 ('education',): 0.17652813987216417,
 ('occupation',): 0.5111860288832916,
 'avg_srmse_1_dims': 0.6595558379637183,
 ('age', 'gender'): 2.607553262099893,
 ('age', 'license'): 2.744209071366722,
 ('age', 'relation'): 4.981531735125941,
 ('age', 'education'): 3.5174736317485946,
 ('age', 'occupation'): 3.7781980374967183,
 ('gender', 'license'): 0.06994081000673404,
 ('gender', 'relation'): 0.5870809512452912,
 ('gender', 'education'): 0.184117808608423,
 ('gender', 'occupation'): 0.5273942781301408,
 ('license', 'relation'): 0.5017532047057485,
 ('license', 'education'): 0.274658991417678,
 ('license', 'occupation'): 0.6175221771018661,
 ('relation', 'education'): 0.7781438191588634,
 ('relation', 'occupation'): 1.1903396952454806,
 ('education', 'occupation'): 0.9400290981487585,
 'avg_srmse_2_dims': 1.553329771440457,
 ('age', 'gender', 'license'

### 将个人集成到家庭中

In [121]:
## 有老人、有学生、有驾照的比例、性别比例、本科以上比例、雇佣比例, 集成到家庭中
total_ls_raw = []
total_ls_generate = []
person_number_raw = raw_family_df['family_家庭成员数量'].values.cumsum()
person_number_generate = results_family_df['family_家庭成员数量'].values.cumsum()
for i in range(len(person_number_raw)):
    if i == 0:
        start_idx = 0
    else:
        start_idx = person_number_raw[i-1]
    end_idx = person_number_raw[i]
    person_in_family = raw_person_df.iloc[start_idx:end_idx,:]
    
    ls = []
    ls.append(int((person_in_family['age'] > 11).sum() >= 1))
    ls.append((person_in_family['license']).mean())
    ls.append((person_in_family['gender']).mean())
    ls.append((person_in_family['education'].isin([6, 7])).mean())
    ls.append((-person_in_family['occupation'].isin([14, 13, 16])).mean())
    total_ls_raw.append(ls)

for i in range(len(person_number_generate)):
    if i == 0:
        start_idx = 0
    else:
        start_idx = person_number_generate[i-1]
    end_idx = person_number_generate[i]
    person_in_family_generate = results_person_df.iloc[start_idx:end_idx,:]

    ls_generate = []
    ls_generate.append(int((person_in_family_generate['age'] > 11).sum() >= 1))
    ls_generate.append((person_in_family_generate['license']).mean())
    ls_generate.append((person_in_family_generate['gender']).mean())
    ls_generate.append((person_in_family_generate['education'].isin([6, 7])).mean())
    ls_generate.append((-person_in_family_generate['occupation'].isin([14, 13, 16])).mean())
    total_ls_generate.append(ls_generate)




In [122]:
extended_raw_family_df = pd.concat([raw_family_df.reset_index(drop=True), pd.DataFrame(total_ls_raw, columns=['have_elder_ext', 'license_ext', 'gender_ext', 'education_ext', 'employed_ext'])], axis=1)
extended_results_family_df = pd.concat([results_family_df.reset_index(drop=True), pd.DataFrame(total_ls_generate, columns=['have_elder_ext', 'license_ext', 'gender_ext', 'education_ext', 'employed_ext'])], axis=1)

In [123]:
extended_raw_family_df['license_ext'] = pd.cut(
    extended_raw_family_df['license_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['license_ext'] = pd.cut(
    extended_results_family_df['license_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['gender_ext'] = pd.cut(
    extended_raw_family_df['gender_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['gender_ext'] = pd.cut(
    extended_results_family_df['gender_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['education_ext'] = pd.cut(
    extended_raw_family_df['education_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['education_ext'] = pd.cut(
    extended_results_family_df['education_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_raw_family_df['employed_ext'] = pd.cut(
    extended_raw_family_df['employed_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

extended_results_family_df['employed_ext'] = pd.cut(
    extended_results_family_df['employed_ext'], 
    bins=np.linspace(0, 1, 6),   # 生成 [0,0.2,0.4,0.6,0.8,1.0]
    labels=False,                # 输出整数标签 0,1,2,3,4
    include_lowest=True          # 包含左边界0
)

In [124]:
max_value_list_extendfamily = []
for i in range(len(extended_raw_family_df.columns)):
    raw_max = extended_raw_family_df.iloc[:, i].max()
    results_max = extended_results_family_df.iloc[:, i].max()
    max_value = max(raw_max, results_max)
    max_value_list_extendfamily.append(max_value)

In [125]:
srmse_results = calc_srmse(extended_raw_family_df, extended_results_family_df, max_value_list_extendfamily)

Average SRMSE for 1-dimensional combinations: 0.7204457651932277
Average SRMSE for 2-dimensional combinations: 1.7513966636696476
Average SRMSE for 3-dimensional combinations: 3.1810221500679527
Average SRMSE for 4-dimensional combinations: 5.049207543062855
Average SRMSE for 5-dimensional combinations: 7.305399461972335
Average SRMSE for 6-dimensional combinations: 9.798614722817769
Average SRMSE for 7-dimensional combinations: 12.33553478533005
Average SRMSE for 8-dimensional combinations: 14.724938161097773
Average SRMSE for 9-dimensional combinations: 16.810456864374345
Average SRMSE for 10-dimensional combinations: 18.494644753708418
Average SRMSE for 11-dimensional combinations: 19.744899747679646
Average SRMSE for 12-dimensional combinations: 20.581734184161096
Average SRMSE for 13-dimensional combinations: 21.05669055978711
Average SRMSE for 14-dimensional combinations: 21.231301225944836
